## This example applies DeepPhasePick on seismic data downloaded using FDSN web service client for ObsPy.

In [1]:
import os
os.chdir('../')
import config, data, model, util                                                
from obspy.clients.fdsn import Client                                           
import obspy.core as oc 

## 1. Configure DPP

In [3]:
# config
util.init_session()
dpp_config = config.Config()
dpp_config.set_trigger(pthres_p=[0.9, 0.001], pthres_s=[0.9, 0.001])
dpp_config.set_picking(op_conds=['1','2','3','4'], tp_th_add=1.5, dt_sp_near=1.5, dt_ps_max=25., dt_sdup_max=2., mcd_iter=10, run_mcd=True)
# dpp_config.set_picking(op_conds=['1','2','3','4'], tp_th_add=1.5, dt_sp_near=1.5, dt_ps_max=25., dt_sdup_max=2., mcd_iter=10, run_mcd=False)
dpp_config.set_data(
    stas=['PB01', 'PB02'],
    net='CX',
    ch='HH',
    archive='sample_data/CX_20140401',
    opath='out_CX_20140401'
)
dpp_config.set_time(
    dt_iter=3600.,
    tstart = "2014-04-01T02:00:00",
    tend = "2014-04-01T03:00:00",    
)

__pycache__/ removed
~/.nv/ not found, continuing...


## 2. Download seismic data and read it into DPP

In [4]:
# download and archive seismic waveforms
client = Client("GFZ")
os.makedirs(f"{dpp_config.data['archive']}", exist_ok=True)
tstart = oc.UTCDateTime(dpp_config.time['tstart'])
tend = oc.UTCDateTime(dpp_config.time['tend'])
#
for sta in dpp_config.data['stas']:
    st = client.get_waveforms(network=dpp_config.data['net'], station=sta, location="*", channel=f"{dpp_config.data['ch']}?", starttime=tstart, endtime=tend)
    # print(st)
    st_name = f"{dpp_config.data['archive']}/{st[0].stats.network}.{st[0].stats.station}..{st[0].stats.channel[:-1]}.mseed"
    print(f"writing stream {st_name}...")
    st.write(st_name, format="MSEED")

writing stream sample_data/CX_20140401/CX.PB01..HH.mseed...
writing stream sample_data/CX_20140401/CX.PB02..HH.mseed...


In [5]:
# data
dpp_data = data.Data()
dpp_data.read_from_directory(dpp_config)
#
# for k in dpp_data.data:
#     print(k, dpp_data.data[k])

reading seismic waveform: sample_data/CX_20140401/CX.PB01..HH.mseed
reading seismic waveform: sample_data/CX_20140401/CX.PB02..HH.mseed
6 Trace(s) in Stream:
CX.PB01..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHE | 2014-04-01T01:59:59.998394Z - 2014-04-01T02:59:59.998394Z | 100.0 Hz, 360001 samples
CX.PB01..HHN | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHE | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHN | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
#
processing raw stream data...
detrend...
resampling...
merging...
#
time windows (1) for iteration over continuous waveforms:
[UTCDateTime(2014, 4, 1, 2, 0), UTCDateTime(2014, 4, 1, 3, 0)]

slicing...
1 {'st': {'PB01': <obspy.core.stream.St

## 3. Run phase detection and picking

In [6]:
# model
dpp_model = model.Model(verbose=False)
#
print(dpp_model.model_detection['best_model'].summary())
print(dpp_model.model_picking_P['best_model'].summary())
print(dpp_model.model_picking_S['best_model'].summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv1d_1 (Separabl (None, 480, 12)           99        
_________________________________________________________________
activation_1 (Activation)    (None, 480, 12)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 480, 12)           48        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 240, 12)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 240, 12)           0         
_________________________________________________________________
separable_conv1d_2 (Separabl (None, 240, 24)           444       
_________________________________________________________________
activation_2 (Activation)    (None, 240, 24)          

In [7]:
# run phase detection
dpp_model.run_detection(dpp_config, dpp_data, save_dets=False, save_data=False)

#
Calculating predictions for stream: CX.PB01..HH?...
strimming stream: 1, 1
720/720 [==============================] - 29s 41ms/step
3 Trace(s) in Stream:
CX.PB01..HHE | 2014-04-01T01:59:59.998394Z - 2014-04-01T02:59:59.998394Z | 100.0 Hz, 360001 samples
CX.PB01..HHN | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 13, s_picks = 8
#
Calculating predictions for stream: CX.PB02..HH?...
720/720 [==============================] - 30s 42ms/step
3 Trace(s) in Stream:
CX.PB02..HHE | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHN | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 10, s_picks = 4


In [8]:
# run phase picking
dpp_model.run_picking(dpp_config, dpp_data, save_plots=True, save_stats=True, save_picks=False)

#
1, 2014-04-01T02:00:00.000000Z, 2014-04-01T03:00:00.000000Z, PB01
triggered picks (P, S): 13, 8
selected picks (P, S): 10, 6
#
P pick: 1/10


100%|██████████| 10/10 [00:16<00:00,  1.66s/it]


3.36 3.57 3.48 3.8 out_CX_20140401/20140401T020000_20140401T030000 PB01
plotting predicted phase P: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB01_P_01.png
plotting predicted phase P: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB01_P_mc_01.png
tpred = 3.570
terr(1 x pb_std) = (-0.090, +0.230)
pick_class = 2
pb, pb_std = (0.510, 0.226)
#
P pick: 2/10


100%|██████████| 10/10 [00:13<00:00,  1.32s/it]


3.36 3.54 3.42 3.7 out_CX_20140401/20140401T020000_20140401T030000 PB01
plotting predicted phase P: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB01_P_02.png
plotting predicted phase P: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB01_P_mc_02.png
tpred = 3.540
terr(1 x pb_std) = (-0.120, +0.160)
pick_class = 2
pb, pb_std = (0.524, 0.167)
#
P pick: 3/10


...

2.4 2.19 2.08 2.34 out_CX_20140401/20140401T020000_20140401T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_01.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_01.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_mc_01.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_mc_01.png
tpred = 2.190
terr(1 x pb_std) = (-0.110, +0.150)
pick_class = 2
pb, pb_std = (0.515, 0.239)
#
S pick: 2/3


100%|██████████| 10/10 [00:09<00:00,  1.10it/s]


2.4 2.39 2.35 2.56 out_CX_20140401/20140401T020000_20140401T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_02.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_02.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_mc_02.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_mc_02.png
tpred = 2.390
terr(1 x pb_std) = (-0.040, +0.170)
pick_class = 2
pb, pb_std = (0.542, 0.192)
#
S pick: 3/3


100%|██████████| 10/10 [00:08<00:00,  1.22it/s]


2.4 2.41 2.22 2.55 out_CX_20140401/20140401T020000_20140401T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_03.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_03.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_mc_03.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_mc_03.png
tpred = 2.410
terr(1 x pb_std) = (-0.190, +0.140)
pick_class = 2
pb, pb_std = (0.523, 0.167)


## 4. Plot predicted phases

In [ ]:
# plots
util.plot_predicted_phases(dpp_config, dpp_data, dpp_model)
# util.plot_predicted_phases(dpp_config, dpp_data, dpp_model, plot_probs=['P','S'], shift_probs=True)